<a href="https://colab.research.google.com/github/Cd881/CSCI166Project/blob/main/166ProjectDDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# === **Cell 1-2:** *Setup & Imports* ===

In [ ]:
!pip install gymnasium[atari,accept-rom-license] stable-baselines3 torch torchvision tensorboard autorom ale-py

  Using cached ale_py-0.11.2.tar.gz (512 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build ale-py


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [137 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-314\ale_py
  copying src\ale\python\env.py -> build\lib.win-amd64-cpython-314\ale_py
  copying src\ale\python\registration.py -> build\lib.win-amd64-cpython-314\ale_py
  copying src\ale\python\vector_env.py -> build\lib.win-amd64-cpython-314\ale_py
  copying src\ale\python\__init__.py -> build\lib.win-amd64-cpython-314\ale_py
  creating build\lib.win-amd64-cpython-314\ale_py\roms
  copying src\ale\python\roms\__init__.py -> build\lib.win-amd64-cpython-314\ale_py\roms
  running egg_info
  writing ale_py.egg-info\PKG-INFO
  writing dependency_links to ale_py.egg-info\dependency_links.txt
  writing requirements to ale_py.egg-info\requires.txt
  writing top-level names to ale_py.egg-info\top_level.txt
  reading manifest file 'ale_py.egg-info\SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license f

Looking in indexes: https://download.pytorch.org/whl/cu130


ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


In [ ]:
# !C:\Users\blu3s\AppData\Local\Programs\Python\Python313\python.exe -m AutoROM --accept-license

import ale_py
import gymnasium as gym
import numpy as np
import collections
import time
from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import typing as tt
from gymnasium import spaces
from stable_baselines3.common import atari_wrappers
from datetime import datetime
import os

import matplotlib.pyplot as plt

print("Torch:", torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Torch: 2.9.1+cpu
Using device: cpu


# === **Cell 3:** *DQN Model* ===

In [ ]:
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )
        size = self.conv(torch.zeros(1, *input_shape)).size()[-1]
        self.fc = nn.Sequential(
            nn.Linear(size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def forward(self, x: torch.ByteTensor):
        x = x.float() / 255.0
        return self.fc(self.conv(x))

# === **Cell 4:** *Custom Wrappers* ===

In [ ]:
class ImageToPyTorch(gym.ObservationWrapper):
    """
    Reorders (H,W,C) -> (C,H,W)
    """
    def __init__(self, env):
        super().__init__(env)
        obs = self.observation_space
        assert len(obs.shape) == 3
        new_shape = (obs.shape[-1], obs.shape[0], obs.shape[1])
        self.observation_space = spaces.Box(
            low=obs.low.min(), high=obs.high.max(), shape=new_shape, dtype=obs.dtype)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class BufferWrapper(gym.ObservationWrapper):
    """
    Frame stacking implemented manually.
    """
    def __init__(self, env, n_steps):
        super().__init__(env)
        obs = env.observation_space
        self.buffer = collections.deque(maxlen=n_steps)
        self.observation_space = spaces.Box(
            obs.low.repeat(n_steps, axis=0),
            obs.high.repeat(n_steps, axis=0),
            dtype=obs.dtype
        )

    def reset(self, *, seed=None, options=None):
        for _ in range(self.buffer.maxlen):
            self.buffer.append(np.zeros_like(self.env.observation_space.low))
        obs, extra = self.env.reset(seed=seed, options=options)
        return self.observation(obs), extra

    def observation(self, observation):
        self.buffer.append(observation)
        return np.concatenate(self.buffer)

# === **Cell 5:** *make_env* ===

In [ ]:
def make_env(env_name: str, n_steps=4, render_mode=None, **kwargs):
    print(f"Creating environment {env_name}")
    env = gym.make(env_name, render_mode=render_mode, **kwargs)
    env = atari_wrappers.AtariWrapper(env, clip_reward=False, noop_max=0)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, n_steps=n_steps)
    return env

# === **Cell 6:** *Base Config* ===

In [ ]:
DEFAULT_ENV_NAME = "ALE/SpaceInvaders-v5"
MEAN_REWARD_BOUND = 1000

--- *Base Settings* ---

In [ ]:
isBaseSettings = True

GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 10000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000
REPLAY_START_SIZE = 10000

SAVE_EPSILON = 0.5
EPSILON_DECAY_LAST_FRAME = 150000
EPSILON_START = 1.0
EPSILON_FINAL = 0.01

 --- *Experimentation Config* ---

In [ ]:
EXPERIMENT_NAME = "Experiment1"
isBaseSettings = False

GAMMA = 0.99
BATCH_SIZE = 64 # Smoother updates
REPLAY_SIZE = 30000 # More diverse transitions
LEARNING_RATE = 2e-4 # Stability
SYNC_TARGET_FRAMES = 2000 # Less oscillation
REPLAY_START_SIZE = 10000

SAVE_EPSILON = 0.5
EPSILON_DECAY_LAST_FRAME = 300000 # Slower decay = more exploration early
EPSILON_START = 1.0
EPSILON_FINAL = 0.01

In [ ]:
EXPERIMENT_NAME = "Experiment2"
isBaseSettings = False

GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 50000         # ↑ larger = more replay benefit
REPLAY_START_SIZE = 20000   # ↑ learn only after bigger replay buffer
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 500    # ↑ more frequent sync improves stability

# Exploration (reduced)
EPSILON_START = 0.5         # ↓ far less early exploration
EPSILON_FINAL = 0.05
EPSILON_DECAY_LAST_FRAME = 100000   # faster decay to exploitation
SAVE_EPSILON = 0.5

In [ ]:
EXPERIMENT_NAME = "FinalRun"

# === **Cell 7:** *Save Paths* ===

In [ ]:
# from google.colab import drive
from pathlib import Path

#drive.mount('/content/drive')
#save_dir_drive = "/content/drive/MyDrive/PUBLIC/166Project"
#os.makedirs(save_dir_drive, exist_ok=True)

save_dir_local = "saved_models"
os.makedirs(save_dir_local, exist_ok=True)

safe_env_name = DEFAULT_ENV_NAME.replace("/", "_")

# === **Cell 8:** *Experience + Buffer* ===

In [ ]:
@dataclass
class Experience:
    state: tt.Any
    action: int
    reward: float
    done_trunc: bool
    new_state: tt.Any


class ExperienceBuffer:
    def __init__(self, capacity: int):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience: Experience):
        self.buffer.append(experience)

    def sample(self, batch_size: int):
        indices = np.random.choice(len(self), batch_size, replace=False)
        return [self.buffer[idx] for idx in indices]

# === **Cell 9:** *Agent* ===

In [ ]:
class Agent:
    def __init__(self, env: gym.Env, exp_buffer: ExperienceBuffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self.state = None
        self._reset()

    def _reset(self):
        self.state, _ = self.env.reset()
        self.total_reward = 0.0

    @torch.no_grad()
    def play_step(self, net: DQN, device: torch.device, epsilon=0.0):
        done_reward = None

        if np.random.random() < epsilon:
            action = self.env.action_space.sample()
        else:
            state_v = torch.as_tensor(self.state).to(device).unsqueeze(0)
            q_vals = net(state_v)
            action = int(torch.argmax(q_vals).item())

        new_state, reward, is_done, is_tr, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(
            state=self.state,
            action=action,
            reward=float(reward),
            done_trunc=(is_done or is_tr),
            new_state=new_state
        )
        self.exp_buffer.append(exp)

        self.state = new_state
        if is_done or is_tr:
            done_reward = self.total_reward
            self._reset()

        return done_reward

# === **Cell 10:** *Batch + Loss* ===

In [ ]:
def batch_to_tensors(batch, device):
    states = torch.as_tensor(np.asarray([e.state for e in batch])).to(device)
    actions = torch.LongTensor([e.action for e in batch]).to(device)
    rewards = torch.FloatTensor([e.reward for e in batch]).to(device)
    dones = torch.BoolTensor([e.done_trunc for e in batch]).to(device)
    next_states = torch.as_tensor(np.asarray([e.new_state for e in batch])).to(device)
    return states, actions, rewards, dones, next_states

def calc_loss(batch: tt.List[Experience], net: DQN, tgt_net: DQN, device: torch.device) -> torch.Tensor:
    states_t, actions_t, rewards_t, dones_t, new_states_t = batch_to_tensors(batch, device)

    q_sa = net(states_t).gather(1, actions_t.unsqueeze(-1)).squeeze(-1)

    # ✅ Double DQN target: choose with net, evaluate with tgt_net
    with torch.no_grad():
        next_actions = net(new_states_t).argmax(dim=1)
        next_q = tgt_net(new_states_t).gather(1, next_actions.unsqueeze(-1)).squeeze(-1)
        next_q[dones_t] = 0.0
        target = rewards_t + GAMMA * next_q

    return nn.MSELoss()(q_sa, target)

# === **Cell 11:** *Initialize Env + Nets* ===

In [ ]:
model_comment = f"test_epsdec{EPSILON_DECAY_LAST_FRAME}_rs{REPLAY_START_SIZE}_sync{SYNC_TARGET_FRAMES}"

env = make_env(DEFAULT_ENV_NAME)
net = DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = DQN(env.observation_space.shape, env.action_space.n).to(device)

if isBaseSettings:
    writer = SummaryWriter(comment=f"-{DEFAULT_ENV_NAME}-{model_comment}")
else:
    writer = SummaryWriter(log_dir=f"runs/{EXPERIMENT_NAME}")
print(net)

buffer = ExperienceBuffer(REPLAY_SIZE)
agent = Agent(env, buffer)

Creating environment ALE/SpaceInvaders-v5
DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
)


# === **Cell 12:** *Training Loop* ===

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_m_reward = None
start_time = time.time()

while True:
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_DECAY_LAST_FRAME)

    reward = agent.play_step(net, device, epsilon)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        ts_frame, ts = frame_idx, time.time()

        m_reward = np.mean(total_rewards[-100:])
        elapsed = (time.time() - start_time) / 60

        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", m_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)

        # Save model if improved
        if best_m_reward is None or m_reward > best_m_reward + SAVE_EPSILON:
            print(f"{frame_idx}: reward {m_reward:.3f}, eps {epsilon:.02f}, {speed:.2f} f/s, time {elapsed:.1f} min")
            timestamp = datetime.now().strftime("%Y%m%d-%H%M")
            filename = f"{safe_env_name}-best_{int(m_reward)}-{timestamp}-{model_comment}.dat"

            # Save to drive and local
            # torch.save(net.state_dict(), os.path.join(save_dir_drive, filename))
            torch.save(net.state_dict(), os.path.join(save_dir_local, filename))

            print("Saved:", filename)
            best_m_reward = m_reward

        if m_reward > MEAN_REWARD_BOUND:
            print("Solved!")
            break

    if len(buffer) < REPLAY_START_SIZE:
        continue

    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict())

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss = calc_loss(batch, net, tgt_net, device)
    loss.backward()
    optimizer.step()

env.close()
writer.close()

71: reward 85.000, eps 0.50, 338.65 f/s, time 0.0 min
Saved: ALE_SpaceInvaders-v5-best_85-20251127-1132-test_epsdec100000_rs20000_sync500.dat
11076: reward 86.400, eps 0.39, 300.57 f/s, time 0.6 min
Saved: ALE_SpaceInvaders-v5-best_86-20251127-1133-test_epsdec100000_rs20000_sync500.dat
18414: reward 87.650, eps 0.32, 295.05 f/s, time 1.0 min
Saved: ALE_SpaceInvaders-v5-best_87-20251127-1133-test_epsdec100000_rs20000_sync500.dat
55764: reward 88.250, eps 0.05, 27.25 f/s, time 21.8 min
Saved: ALE_SpaceInvaders-v5-best_88-20251127-1154-test_epsdec100000_rs20000_sync500.dat
55807: reward 89.050, eps 0.05, 27.31 f/s, time 21.8 min
Saved: ALE_SpaceInvaders-v5-best_89-20251127-1154-test_epsdec100000_rs20000_sync500.dat
55866: reward 89.650, eps 0.05, 25.83 f/s, time 21.9 min
Saved: ALE_SpaceInvaders-v5-best_89-20251127-1154-test_epsdec100000_rs20000_sync500.dat
56000: reward 90.250, eps 0.05, 27.34 f/s, time 22.0 min
Saved: ALE_SpaceInvaders-v5-best_90-20251127-1154-test_epsdec100000_rs20000_

KeyboardInterrupt: 

# === **Cells 13:** *Learning Curve Plot* ===

In [ ]:
!kill 3156

'kill' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 3156), started 0:01:17 ago. (Use '!kill 3156' to kill it.)